<a href="https://colab.research.google.com/github/xiaochengJF/DeepLearning/blob/master/Faster_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font face=STCAIYUN size=10 color=purple>构建Faster RCNN</font>
<font face=楷体 color=skyblue>

- Region Proposal network (RPN)  
- RPN loss functions  
- Region of Interest Pooling (ROI)  
- ROI loss functions
</font>

**<font face=楷体 color=skyblue size=5>训练Faster RCNN时通常的数据流</font>**

<font face=楷体>1、从图像中提取特征  
2、产生anchor目标  
3、RPN网络中得到位置和目标预测分值  
4、取前N个坐标及其目标得分即建议层  
5、传递前N个坐标通过Fast R-CNN网络，生成4中建议的每个位置的位置和cls预测  
6、对4中建议的每个坐标生成建议目标  
7、采用2,3计算rpn_cls_loss和rpn_reg_loss  
8、采用5,6计算roi_cls_loss和roi_reg_loss  
</font>  

<font face=楷体 color=skyblue>定义一张图像、两个bbox及其标签</font>

In [0]:
import torch
image = torch.zeros((1, 3, 800, 800)).float()

bbox = torch.FloatTensor([[20, 30, 400, 500], [300, 400, 500, 600]]) # [y1, x1, y2, x2] format
labels = torch.LongTensor([6, 8]) # 0 represents background
sub_sample = 16

<font face=楷体 color=skyblue>生成一个dummy image并且设置volatile为False</font>



In [0]:
import torchvision
dummy_img = torch.zeros((1, 3, 800, 800)).float()
print(dummy_img.shape)
#Out: torch.Size([1, 3, 800, 800])

torch.Size([1, 3, 800, 800])


<font face=楷体 color=skyblue>列出VGG16的所有层</font>

In [0]:
model = torchvision.models.vgg16(pretrained=True)
fe = list(model.features)
# print(fe) # length is 15
for layer in fe:
    print (layer)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth



Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=Tru

<font face=楷体 color=skyblue>将图像喂给网络，确定得到相应的尺寸</font>

In [0]:
req_features = []
k = dummy_img.clone()
for i in fe:
    k = i(k)
    if k.size()[2] < 800//16:
        break
    req_features.append(i)
    out_channels = k.size()[1]
print(len(req_features)) #30
print(out_channels) # 512

30
512


<font face=楷体 color=skyblue>将list转换为Sequential module</font>

In [0]:
faster_rcnn_fe_extractor = torch.nn.Sequential(*req_features)

<font face=楷体 color=skyblue>用构建好的VGG16网络提取特征图</font>

In [0]:
out_map = faster_rcnn_fe_extractor(image)
print(out_map.size())

torch.Size([1, 512, 50, 50])


## Anchor boxes
<font face=楷体>

- 在所有feature map的坐标上生成Anchor
- 对每个目标分配标签及坐标（相对于anchor）
- 在feature map坐标生成Anchor  

采用anchor_scales=[8，16，32] ratio=[0.5，1，2] subsampling=16
</font>

<font face=楷体 color=skyblue>feature map的一个像素位置生成9个anchor boxes，每个像素对应输入图像中的16*16像素</font>

In [0]:
import numpy as np
ratios = [0.5, 1, 2]
anchor_scales = [8, 16, 32]

anchor_base = np.zeros((len(ratios) * len(anchor_scales), 4), dtype=np.float32)

print(anchor_base)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [0]:
ctr_y = sub_sample / 2.
ctr_x = sub_sample / 2.
print(ctr_y, ctr_x)

for i in range(len(ratios)):
 for j in range(len(anchor_scales)):
   h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])    # 输出的feature map的每个像素对应图像中的16*16像素
   w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])  # ration决定了anchor的长宽比例(同面积)

   index = i * len(anchor_scales) + j

   anchor_base[index, 0] = ctr_y - h / 2.
   anchor_base[index, 1] = ctr_x - w / 2.
   anchor_base[index, 2] = ctr_y + h / 2.
   anchor_base[index, 3] = ctr_x + w / 2.
print(anchor_base)

8.0 8.0
[[ -37.254833  -82.50967    53.254833   98.50967 ]
 [ -82.50967  -173.01933    98.50967   189.01933 ]
 [-173.01933  -354.03867   189.01933   370.03867 ]
 [ -56.        -56.         72.         72.      ]
 [-120.       -120.        136.        136.      ]
 [-248.       -248.        264.        264.      ]
 [ -82.50967   -37.254833   98.50967    53.254833]
 [-173.01933   -82.50967   189.01933    98.50967 ]
 [-354.03867  -173.01933   370.03867   189.01933 ]]


<font face=楷体 color=skyblue>feature map的每一个像素位置都要生成9个anchor boxes，每个像素对应输入图像中的16*16像素</font>

In [0]:
fe_size = (800//16)
ctr_x = np.arange(16, (fe_size+1) * 16, 16)
ctr_y = np.arange(16, (fe_size+1) * 16, 16)
print (ctr_x)

[ 16  32  48  64  80  96 112 128 144 160 176 192 208 224 240 256 272 288
 304 320 336 352 368 384 400 416 432 448 464 480 496 512 528 544 560 576
 592 608 624 640 656 672 688 704 720 736 752 768 784 800]


<font face=楷体 color=skyblue>将feature map每个像素位置作为锚点并映射回输入图像中</font>

In [0]:
index = 0
ctr  = np.zeros(shape=(len(ctr_x)*len(ctr_y),2))
for x in range(len(ctr_x)):
   for y in range(len(ctr_y)):
       ctr[index, 1] = ctr_x[x] - 8  # 前面生成中心点位置偏移了8个像素点
       ctr[index, 0] = ctr_y[y] - 8
       index +=1

In [0]:
anchors = np.zeros(shape=(fe_size * fe_size * 9, 4))  # 共有2500个锚点，每个锚点9个框，每个框4个参数
index = 0
for c in ctr:
 ctr_y, ctr_x = c
 for i in range(len(ratios)):
   for j in range(len(anchor_scales)):
     h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
     w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])
     anchors[index, 0] = ctr_y - h / 2.
     anchors[index, 1] = ctr_x - w / 2.
     anchors[index, 2] = ctr_y + h / 2.
     anchors[index, 3] = ctr_x + w / 2.
     index += 1
print(anchors.shape)
#Out: [22500, 4]

(22500, 4)


<font face=楷体 color=skyblue size=5>给anchor分配标签和位置</font>  
<font face=楷体>
a)与ground-truth-box IoU最大的anchor标记为正标签  
b)与ground-truth-box IoU大于0.7的anchor标记为正标签  
c)与ground-truth-box IoU小于0.3的anchor标记为负标签  
d)其余anchor既不是正样本的也不是负样本，对训练没有帮助  
<font color=red>注意：</font>单个ground-truth对象可以为多个anchor分配正标签  
</font>
<font face=楷体 color=skyblue>通过如下方式对anchor boxes分配标签和位置：</font>   
<font face=楷体>
1、找到有效的anchor boxes的索引，并且生成索引数组，生成标签数组其形状索引数组填充-1（无效anchor boxes，对应上文说的处在边框外的anchor boxes）。  
2、检查是否满足以上a、b、c条件中的一条，并相应填写标签。如果是正anchor box(标签为1)，注意哪个ground-truth目标可以得到这个结果。  
3、计算与anchor box相关的ground-truth的位置(loc)。  
4、通过为所有无效的anchor box填充-1和为所有有效锚箱计算的值来重新组织所有anchor box。  
5、输出应该是(N, 1)数组的标签和带有(N, 4)数组的locs。  
6、找到所有有效anchor boxes的索引：
</font>

In [0]:
bbox = np.asarray([[20, 30, 400, 500], [300, 400, 500, 600]], dtype=np.float32) # [y1, x1, y2, x2] format
labels = np.asarray([6, 8], dtype=np.int8) # 0 represents background 

In [0]:
inside_index = np.where(
       (anchors[:, 0] >= 0) &
       (anchors[:, 1] >= 0) &
       (anchors[:, 2] <= 800) &
       (anchors[:, 3] <= 800)
   )[0]
print(inside_index.shape)
#Out: (8940,)

(8940,)


In [0]:
label = np.empty((len(inside_index), ), dtype=np.int32)
label.fill(-1)
print(label.shape)
#Out = (8940, )

(8940,)


In [0]:
valid_anchor_boxes = anchors[inside_index]
print(valid_anchor_boxes.shape)
#Out = (8940, 4)

(8940, 4)


In [0]:
ious = np.empty((len(valid_anchor_boxes), 2), dtype=np.float32)
ious.fill(0)
print(bbox)
for num1, i in enumerate(valid_anchor_boxes):
   ya1, xa1, ya2, xa2 = i 
   anchor_area = (ya2 - ya1) * (xa2 - xa1)
   for num2, j in enumerate(bbox):
       yb1, xb1, yb2, xb2 = j
       box_area = (yb2- yb1) * (xb2 - xb1)

       inter_x1 = max([xb1, xa1])
       inter_y1 = max([yb1, ya1])
       inter_x2 = min([xb2, xa2])
       inter_y2 = min([yb2, ya2])
       if (inter_x1 < inter_x2) and (inter_y1 < inter_y2):
           iter_area = (inter_y2 - inter_y1) * \
(inter_x2 - inter_x1)
           iou = iter_area / \
(anchor_area+ box_area - iter_area) 
       else:
           iou = 0.

       ious[num1, num2] = iou
print(ious.shape)
#Out: [22500, 2]

[[ 20.  30. 400. 500.]
 [300. 400. 500. 600.]]
(8940, 2)


<font face=楷体 color=skyblue>找到与每个gt_box iou最高的 anchor box</font> 

In [0]:
gt_argmax_ious = ious.argmax(axis=0)
print(gt_argmax_ious)
gt_max_ious = ious[gt_argmax_ious, np.arange(ious.shape[1])]
print(gt_max_ious)
# Out:
# [2262 5620]
# [0.68130493 0.61035156]

[2262 5620]
[0.68130493 0.61035156]


<font face=楷体 color=skyblue>得到每个anchor box与ground-truth box 的最高iou</font> 


In [0]:
argmax_ious = ious.argmax(axis=1)
print(argmax_ious.shape)
print(argmax_ious)
max_ious = ious[np.arange(len(inside_index)), argmax_ious]
print(max_ious)
# Out:
# (22500,)
# [0, 1, 0, ..., 1, 0, 0]
# [0.06811669 0.07083762 0.07083762 ... 0.         0.         0.        ]

(8940,)
[0 0 0 ... 0 0 0]
[0.06811669 0.07083762 0.07083762 ... 0.         0.         0.        ]


In [0]:
gt_argmax_ious = np.where(ious == gt_max_ious)[0]
print(gt_argmax_ious)
# Out:
# [2262, 2508, 5620, 5628, 5636, 5644, 5866, 5874, 5882, 5890, 6112,
#        6120, 6128, 6136, 6358, 6366, 6374, 6382]

[2262 2508 5620 5628 5636 5644 5866 5874 5882 5890 6112 6120 6128 6136
 6358 6366 6374 6382]


In [0]:
print(np.where(ious == gt_max_ious))

(array([2262, 2508, 5620, 5628, 5636, 5644, 5866, 5874, 5882, 5890, 6112,
       6120, 6128, 6136, 6358, 6366, 6374, 6382]), array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))


In [0]:
pos_iou_threshold  = 0.7
neg_iou_threshold = 0.3

In [0]:
label[max_ious < neg_iou_threshold] = 0

In [0]:
label[gt_argmax_ious] = 1

In [0]:
label[max_ious >= pos_iou_threshold] = 1

In [0]:
pos_ratio = 0.5
n_sample = 256

In [0]:
n_pos = pos_ratio * n_sample

In [0]:
pos_index = np.where(label == 1)[0]
if len(pos_index) > n_pos:
   disable_index = np.random.choice(pos_index, size=(len(pos_index) - n_pos), replace=False)
   label[disable_index] = -1

In [0]:
n_neg = n_sample * np.sum(label == 1)
neg_index = np.where(label == 0)[0]
if len(neg_index) > n_neg:
   disable_index = np.random.choice(neg_index, size=(len(neg_index) - n_neg), replace = False)
   label[disable_index] = -1

In [0]:
max_iou_bbox = bbox[argmax_ious]
print(max_iou_bbox)
#Out
# [[ 20.,  30., 400., 500.],
#  [ 20.,  30., 400., 500.],
#  [ 20.,  30., 400., 500.],
#  ...,
#  [ 20.,  30., 400., 500.],
#  [ 20.,  30., 400., 500.],
#  [ 20.,  30., 400., 500.]]

tensor([[ 20.,  30., 400., 500.],
        [ 20.,  30., 400., 500.],
        [ 20.,  30., 400., 500.],
        ...,
        [ 20.,  30., 400., 500.],
        [ 20.,  30., 400., 500.],
        [ 20.,  30., 400., 500.]])


In [0]:
height = valid_anchor_boxes[:, 2] - valid_anchor_boxes[:, 0]
width = valid_anchor_boxes[:, 3] - valid_anchor_boxes[:, 1]
ctr_y = valid_anchor_boxes[:, 0] + 0.5 * height
ctr_x = valid_anchor_boxes[:, 1] + 0.5 * width
base_height = max_iou_bbox[:, 2] - max_iou_bbox[:, 0]
base_width = max_iou_bbox[:, 3] - max_iou_bbox[:, 1]
base_ctr_y = max_iou_bbox[:, 0] + 0.5 * base_height
base_ctr_x = max_iou_bbox[:, 1] + 0.5 * base_width

In [0]:
eps = np.finfo(height.dtype).eps
height = np.maximum(height, eps)
width = np.maximum(width, eps)
dy = (base_ctr_y - ctr_y) / height
dx = (base_ctr_x - ctr_x) / width
dh = np.log(base_height / height)
dw = np.log(base_width / width)
anchor_locs = np.vstack((dy, dx, dh, dw)).transpose()
print(anchor_locs)
#Out:
# [[ 0.5855727   2.3091455   0.7415673   1.647276  ]
#  [ 0.49718437  2.3091455   0.7415673   1.647276  ]
#  [ 0.40879607  2.3091455   0.7415673   1.647276  ]
#  ...
#  [-2.50802    -5.292254    0.7415677   1.6472763 ]
#  [-2.5964084  -5.292254    0.7415677   1.6472763 ]
#  [-2.6847968  -5.292254    0.7415677   1.6472763 ]]

[[ 0.5855728   2.30914558  0.7415674   1.64727602]
 [ 0.49718446  2.30914558  0.7415674   1.64727602]
 [ 0.40879611  2.30914558  0.7415674   1.64727602]
 ...
 [-2.50801936 -5.29225232  0.7415674   1.64727602]
 [-2.59640771 -5.29225232  0.7415674   1.64727602]
 [-2.68479606 -5.29225232  0.7415674   1.64727602]]


In [0]:
anchor_labels = np.empty((len(anchors),), dtype=label.dtype)
anchor_labels.fill(-1)
anchor_labels[inside_index] = label

In [0]:
anchor_locations = np.empty((len(anchors),) + anchors.shape[1:], dtype=anchor_locs.dtype)
anchor_locations.fill(0)
anchor_locations[inside_index, :] = anchor_locs
print(anchor_locs.shape)
print(anchor_locations.shape)

(8940, 4)
(22500, 4)


In [0]:
import torch.nn as nn
mid_channels = 512
in_channels = 512 # depends on the output feature map. in vgg 16 it is equal to 512
n_anchor = 9 # Number of anchors at each location
conv1 = nn.Conv2d(in_channels, mid_channels, 3, 1, 1)
reg_layer = nn.Conv2d(mid_channels, n_anchor *4, 1, 1, 0)
cls_layer = nn.Conv2d(mid_channels, n_anchor *2, 1, 1, 0) ## I will be going to use softmax here. you can equally use sigmoid if u replace 2 with 1.

In [0]:
# conv sliding layer
conv1.weight.data.normal_(0, 0.01)
conv1.bias.data.zero_()

# Regression layer
reg_layer.weight.data.normal_(0, 0.01)
reg_layer.bias.data.zero_()

# classification layer
cls_layer.weight.data.normal_(0, 0.01)
cls_layer.bias.data.zero_()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
x = conv1(out_map) # out_map is obtained in section 1
pred_anchor_locs = reg_layer(x)
pred_cls_scores = cls_layer(x)

print(pred_cls_scores.shape, pred_anchor_locs.shape)

#Out:
#torch.Size([1, 18, 50, 50]) torch.Size([1, 36, 50, 50])

torch.Size([1, 18, 50, 50]) torch.Size([1, 36, 50, 50])


In [0]:
pred_anchor_locs = pred_anchor_locs.permute(0, 2, 3, 1).contiguous().view(1, -1, 4)
print(pred_anchor_locs.shape)

#Out: torch.Size([1, 22500, 4])

pred_cls_scores = pred_cls_scores.permute(0, 2, 3, 1).contiguous()
print(pred_cls_scores.shape)
#Out torch.Size([1, 50, 50, 18])

objectness_score = pred_cls_scores.view(1, 50, 50, 9, 2)[:, :, :, :, 1].contiguous().view(1, -1)
print(objectness_score.shape)
#Out torch.Size([1, 22500])

pred_cls_scores  = pred_cls_scores.view(1, -1, 2)
print(pred_cls_scores.shape)
# Out torch.size([1, 22500, 2])

torch.Size([1, 22500, 4])
torch.Size([1, 50, 50, 18])
torch.Size([1, 22500])
torch.Size([1, 22500, 2])


In [0]:
nms_thresh = 0.7
n_train_pre_nms = 12000
n_train_post_nms = 2000
n_test_pre_nms = 6000
n_test_post_nms = 300
min_size = 16

In [0]:
anc_height = anchors[:, 2] - anchors[:, 0]
anc_width = anchors[:, 3] - anchors[:, 1]
anc_ctr_y = anchors[:, 0] + 0.5 * anc_height
anc_ctr_x = anchors[:, 1] + 0.5 * anc_width

In [0]:
pred_anchor_locs_numpy = pred_anchor_locs[0].data.numpy()
objectness_score_numpy = objectness_score[0].data.numpy()

dy = pred_anchor_locs_numpy[:, 0::4]
dx = pred_anchor_locs_numpy[:, 1::4]
dh = pred_anchor_locs_numpy[:, 2::4]
dw = pred_anchor_locs_numpy[:, 3::4]

ctr_y = dy * anc_height[:, np.newaxis] + anc_ctr_y[:, np.newaxis]
ctr_x = dx * anc_width[:, np.newaxis] + anc_ctr_x[:, np.newaxis]
h = np.exp(dh) * anc_height[:, np.newaxis]
w = np.exp(dw) * anc_width[:, np.newaxis]

In [0]:
roi = np.zeros(pred_anchor_locs_numpy.shape, dtype=anchor_locs.dtype)
roi[:, 0::4] = ctr_y - 0.5 * h
roi[:, 1::4] = ctr_x - 0.5 * w
roi[:, 2::4] = ctr_y + 0.5 * h
roi[:, 3::4] = ctr_x + 0.5 * w
print(roi)
#Out:
# [[ -36.897102,  -80.29519 ,   54.09939 ,  100.40507 ],
#  [ -83.12463 , -165.74298 ,   98.67854 ,  188.6116  ],
#  [-170.7821  , -378.22214 ,  196.20844 ,  349.81198 ],
#  ...,
#  [ 696.17816 ,  747.13306 ,  883.4582  ,  836.77747 ],
#  [ 621.42114 ,  703.0614  ,  973.04626 ,  885.31226 ],
#  [ 432.86267 ,  622.48926 , 1146.7059  ,  982.9209  ]]

[[ -37.58030545  -77.12890932   51.77519976  100.36579025]
 [ -78.45750189 -159.36509228   97.96676213  200.90674064]
 [-167.77735559 -325.70108395  187.31676734  399.37995111]
 ...
 [ 697.84474493  746.07445935  879.44257574  840.36405632]
 [ 617.05210063  697.29438059  975.66175356  876.61405353]
 [ 429.31384825  615.78105249 1143.53028084  984.0958582 ]]


In [0]:
img_size = (800, 800) #Image size
roi[:, slice(0, 4, 2)] = np.clip(
           roi[:, slice(0, 4, 2)], 0, img_size[0])
roi[:, slice(1, 4, 2)] = np.clip(
   roi[:, slice(1, 4, 2)], 0, img_size[1])

print(roi)

#Out:
# [[  0.     ,   0.     ,  54.09939, 100.40507],
#  [  0.     ,   0.     ,  98.67854, 188.6116 ],
#  [  0.     ,   0.     , 196.20844, 349.81198],
#  ...,
#  [696.17816, 747.13306, 800.     , 800.     ],
#  [621.42114, 703.0614 , 800.     , 800.     ],
#  [432.86267, 622.48926, 800.     , 800.     ]]

[[  0.           0.          51.77519976 100.36579025]
 [  0.           0.          97.96676213 200.90674064]
 [  0.           0.         187.31676734 399.37995111]
 ...
 [697.84474493 746.07445935 800.         800.        ]
 [617.05210063 697.29438059 800.         800.        ]
 [429.31384825 615.78105249 800.         800.        ]]


In [0]:
hs = roi[:, 2] - roi[:, 0]
ws = roi[:, 3] - roi[:, 1]
keep = np.where((hs >= min_size) & (ws >= min_size))[0]
roi = roi[keep, :]
score = objectness_score_numpy[keep]

print(score.shape)
#Out:
##(22500, ) all the boxes have minimum size of 16

(22500,)


In [0]:
order = score.ravel().argsort()[::-1]
print(order)

#Out:
#[ 889,  929, 1316, ...,  462,  454,    4]

[  450   439   459 ...   466   451 21611]


In [0]:
order = order[:n_train_pre_nms]
roi = roi[order, :]

print(roi.shape)
print(roi)

#Out
# (12000, 4)
# [[607.93866,   0.     , 800.     , 113.38187],
#  [  0.     ,   0.     , 235.29704, 369.64795],
#  [572.177  ,   0.     , 800.     , 373.0086 ],
#  ...,
#  [250.07968, 186.61633, 434.6356 , 276.70615],
#  [490.07974, 154.6163 , 674.6356 , 244.70615],
#  [266.07968, 602.61633, 450.6356 , 692.7062 ]]

(12000, 4)
[[  0.           0.          50.4412434  115.72485537]
 [586.32632826   0.         800.          97.59094087]
 [  0.           0.          67.32273412 119.75210914]
 ...
 [515.26549149  59.01612496 800.         563.49750924]
 [515.26549149 315.01612496 800.         800.        ]
 [515.26549149 123.01612496 800.         627.49750924]]


In [0]:
y1 = roi[:, 0]
x1 = roi[:, 1]
y2 = roi[:, 2]
x2 = roi[:, 3]

area = (x2 - x1 + 1) * (y2 - y1 + 1)
print(order.shape)
order = score.argsort()[::-1]  
print(order.shape)

keep = []

while order.size > 0:
   i = order[0]
   xx1 = np.maximum(x1[i], x1[order[1:]])
   yy1 = np.maximum(y1[i], y1[order[1:]])
   xx2 = np.minimum(x2[i], x2[order[1:]])
   yy2 = np.minimum(y2[i], y2[order[1:]])
  
   w = np.maximum(0.0, xx2 - xx1 + 1)
   h = np.maximum(0.0, yy2 - yy1 + 1)    
   inter = w * h
   ovr = inter / (areas[i] + areas[order[1:]] - inter)
   
   inds = np.where(ovr <= thresh)[0]
   order = order[inds + 1]

keep = keep[:n_train_post_nms] # while training/testing , use accordingly
roi = roi[keep] # the final region proposals

(12000,)
(22500,)


IndexError: ignored

In [0]:
n_sample = 128
pos_ratio = 0.25
pos_iou_thresh = 0.5
neg_iou_thresh_hi = 0.5
neg_iou_thresh_lo = 0.0

In [0]:
ious = np.empty((len(roi), 2), dtype=np.float32)
ious.fill(0)
for num1, i in enumerate(roi):
   ya1, xa1, ya2, xa2 = i  
   anchor_area = (ya2 - ya1) * (xa2 - xa1)
   for num2, j in enumerate(bbox):
       yb1, xb1, yb2, xb2 = j
       box_area = (yb2- yb1) * (xb2 - xb1)

       inter_x1 = max([xb1, xa1])
       inter_y1 = max([yb1, ya1])
       inter_x2 = min([xb2, xa2])
       inter_y2 = min([yb2, ya2])

       if (inter_x1 < inter_x2) and (inter_y1 < inter_y2):
           iter_area = (inter_y2 - inter_y1) * \
(inter_x2 - inter_x1)
           iou = iter_area / (anchor_area+ \
box_area - iter_area)            
       else:
           iou = 0.

       ious[num1, num2] = iou
print(ious.shape)

#Out:
#[1535, 2]

(12000, 2)


In [0]:
gt_assignment = iou.argmax(axis=1)
max_iou = iou.max(axis=1)
print(gt_assignment)
print(max_iou)

#Out:
# [0, 0, 0 ... 1, 1, 0]
# [0.016, 0., 0. ... 0.08034518, 0.10739268, 0.]

In [0]:
gt_roi_label = labels[gt_assignment]
print(gt_roi_label)
#Out:
#[6, 6, 6, ..., 8, 8, 6] q  

In [0]:
pos_index = np.where(max_iou >= pos_iou_thresh)[0]
pos_roi_per_this_image = int(min(pos_roi_per_image, pos_index.size))
if pos_index.size > 0:
   pos_index = np.random.choice(
       pos_index, size=pos_roi_per_this_image, replace=False)
print(pos_roi_per_this_image)
print(pos_index)

#Out
# 18
# [ 257  296  317 1075 1077 1169 1213 1258 1322 1325 1351 1378 1380 1425
#  1472 1482 1489 1495]
